In [26]:
import pandas as pd

kg_ori = pd.read_csv('./KG_processed.csv')

/tmp/ipykernel_46857/52417269.py:3: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  kg_ori = pd.read_csv('./KG_processed.csv')


In [27]:
kg_ori

x_type       x_id      relation        y_type         y_id  \
0        molecule    23978.0  drug_protein  gene/protein           F8   
1        molecule    23978.0  drug_protein  gene/protein           F5   
2        molecule      977.0  drug_protein  gene/protein         HBA2   
3        molecule    82153.0  drug_protein  gene/protein     SERPINA6   
4        molecule  5311000.0  drug_protein  gene/protein     SERPINA6   
...           ...        ...           ...           ...          ...   
2523862  molecule   439153.0    in_pathway       pathway  PWID1228018   
2523863  molecule   644102.0    in_pathway       pathway  PWID1239777   
2523864  molecule      668.0    in_pathway       pathway  PWID1324467   
2523865  molecule   644102.0    in_pathway       pathway  PWID1234075   
2523866  molecule     6176.0    in_pathway       pathway  PWID1234444   

         value_level  x_idx    y_idx  
0                 F8  33625   5390.0  
1                 F5  33625   5388.0  
2               HBA2  86664   6360.0  
3           SERPINA6  78526  11017.0  
4           SERPINA6  54050  11017.0  
...              ...    ...      ...  
2523862  PWID1228018  45820   9216.0  
2523863  PWID1239777  65691  20975.0  
2523864  PWID1324467  67814  31394.0  
2523865  PWID1234075  65691  15274.0  
2523866  PWID1234444  63352  15643.0  

[2523867 rows x 8 columns]

## KG triples

In [29]:
from tqdm import tqdm

kg_triples = ""

for i in tqdm(range(len(kg_ori))):
    head = kg_ori.x_type[i] + '_' + str(kg_ori.x_id[i])
    relation = kg_ori.relation[i]
    if kg_ori.y_type[i] == 'value':
        tail = kg_ori.y_type[i] + '_' + str(kg_ori.y_id[i]).split('.')[0]
    else:
        tail = kg_ori.y_type[i] + '_' + str(kg_ori.y_id[i])

    kg_triples += head + '\t' + relation + '\t' + tail + '\n'

with open('./graph.txt', 'w') as f:
    f.write(kg_triples)



100%|██████████| 2523867/2523867 [01:18<00:00, 31986.05it/s]


In [31]:
entity_set = set()

triples = kg_triples.split('\n')

for triple in triples:
    if triple == '':
        continue
    head, relation, tail = triple.split('\t')
    entity_set.add(head)
    entity_set.add(tail)

len(entity_set)

184819

In [39]:
entity_type = {
    'molecule': 0,
    'gene/protein': 1,
    'disease': 2,
    'effect/phenotype': 3,
    'drug': 4,
    'pathway': 5,
    'value_1': 6,
    'value_2': 7,
    'value_3': 8,
    'value_4': 9,
    'value_5': 10,
    'value_6': 11,
    'value_7': 12,
    'value_8': 13,
    'value_9': 14,
    'value_10': 15,
}

In [36]:
import json

with open('./id2entity.json', 'r') as f:
    id2entity = json.load(f)
with open('./id2relation.json', 'r') as f:
    id2relation = json.load(f)

entity2id = {value: key for key, value in id2entity.items()}
relation2id = {value: key for key, value in id2relation.items()}

with open('./entity2id.json', 'w') as f:
    json.dump(entity2id, f, indent=6)
with open('./relation2id.json', 'w') as f:
    json.dump(relation2id, f, indent=6)

In [42]:
import numpy as np

entity_type_onehot = {}
for i in tqdm(range(len(kg_ori))):
    head = kg_ori.x_type[i] + '_' + str(kg_ori.x_id[i])
    relation = kg_ori.relation[i]
    if kg_ori.y_type[i] == 'value':
        tail = kg_ori.y_type[i] + '_' + str(kg_ori.y_id[i]).split('.')[0]
    else:
        tail = kg_ori.y_type[i] + '_' + str(kg_ori.y_id[i])

    if head not in entity_type_onehot:
        entity_type_onehot[head] = np.zeros(16)
        entity_type_onehot[head][entity_type[kg_ori.x_type[i]]] = 1

    if tail not in entity_type_onehot:
        entity_type_onehot[tail] = np.zeros(16)
        if kg_ori.y_type[i] == 'value':
            entity_type_onehot[tail][entity_type['value_' + str(kg_ori.value_level[i]).split('.')[0]]] = 1
        else:
            entity_type_onehot[tail][entity_type[kg_ori.y_type[i]]] = 1



100%|██████████| 2523867/2523867 [01:21<00:00, 31086.17it/s]


## entity type one-hot label (multi-class)

In [43]:
etype_onehot = np.zeros((len(entity_type_onehot), 16))

for entity, onehot in tqdm(entity_type_onehot.items()):
    etype_onehot[int(entity2id[entity])] = onehot

np.save('./pretrain_data/ent_type_onehot.npy', etype_onehot)


100%|██████████| 184819/184819 [00:00<00:00, 906678.06it/s]


In [59]:
valid_smile = set()
valid_cid_ = set()

with open('./downstream_smile2cid.json', 'r') as f:
    smile2cid = json.load(f)

for smile, cid in tqdm(smile2cid.items()):
    molecule = f"molecule_{cid}.0"
    if cid != None and molecule in entity2id:
        valid_smile.add(smile)
        valid_cid_.add(cid)

100%|██████████| 53263/53263 [00:00<00:00, 828771.70it/s]


In [62]:
vs = 'smiles\n'
for smile in valid_smile:
    vs += smile + '\n'
with open('./valid_smiles.csv', 'w') as f:
    f.write(vs)

## motif labels for valid smiles

In [63]:
# used grover's motif labler to label the motifs
motifs = np.load('./motifs.npz')['features']
valid_smile = vs.split('\n')[1:-1]

In [66]:
id2motifs = {}
for i in range(len(motifs)):
    idx = entity2id['molecule_' + str(smile2cid[valid_smile[i]]) + '.0']
    id2motifs[idx] = motifs[i].tolist()

with open('./pretrain_data/id2motifs.json', 'w') as f:
    json.dump(id2motifs, f, indent=6)


## networkx knowledge graph

In [82]:
import networkx as nx

G = nx.Graph()
triple_set = set()
with open('./graph.txt', 'r') as f:
    lines = f.readlines()

for line in tqdm(lines):
    items = line.split('\t')
    if len(items) == 3:
        h, r, t = items
        t = t[:-1]
        h = int(entity2id[h])
        r = int(relation2id[r])
        t = int(entity2id[t])
        triple = (h, r, t)
        if triple not in triple_set:
            edge = (h, t)
            G.add_edge(*edge, relation=r)
            triple_set.add(triple)

100%|██████████| 2523867/2523867 [00:08<00:00, 283214.79it/s]


In [81]:
nx.write_gpickle(G, './pretrain_data/graph.gpickle')